<a href="https://colab.research.google.com/github/Zinwaiyan274/Waste-Classification/blob/main/Residual_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint
import os
import cv2
import glob
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
num_class = 5

model = Sequential([ResNet50(include_top=False,
                             weights='imagenet',
                             pooling='avg'),
                    Dense(num_class, activation='softmax')])

model.layers[0].trainable = False

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
lr = 0.01
momentum = 0.001
opt = SGD(learning_rate=lr, momentum=momentum)

model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 5)                 10245     
                                                                 
Total params: 23,597,957
Trainable params: 10,245
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
data_path = "/content/drive/MyDrive/II/Wai/Data set"




data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

img_shape = (64,128)

train_gen = data_generator.flow_from_directory(data_path + '/Train',
                                               target_size=(64,128),
                                               batch_size=64,
                                               class_mode='categorical')

val_gen = data_generator.flow_from_directory(data_path + '/val',
                                             target_size=(64,128),
                                             batch_size=64,
                                             class_mode='categorical',
                                             shuffle=False)

test_gen = data_generator.flow_from_directory(data_path + '/Test',
                                             target_size=(64,128),
                                             batch_size=64,
                                             class_mode='categorical',
                                             shuffle=False)

Found 5420 images belonging to 5 classes.
Found 1241 images belonging to 5 classes.
Found 1367 images belonging to 5 classes.


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
n_epoch = 20

model_name = 'resnet50_batch64_sgd01m001'
checkpoint = ModelCheckpoint('./' +  model_name + '.h5',
                             monitor='val_loss',
                             save_best_only=True,
                             verbose=1)



In [ ]:
history = model.fit_generator(train_gen,
                              steps_per_epoch=train_gen.samples/train_gen.batch_size,
                              validation_data=val_gen,
                              validation_steps=val_gen.samples/val_gen.batch_size,
                              epochs = n_epoch,
                              callbacks= [checkpoint])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/20
85/84 [==============================] - ETA: -4s - loss: 0.6712 - accuracy: 0.7983
Epoch 1: val_loss improved from inf to 4.27529, saving model to ./resnet50_batch64_sgd01m001.h5
84/84 [==============================] - 1583s 19s/step - loss: 0.6712 - accuracy: 0.7983 - val_loss: 4.2753 - val_accuracy: 0.2732
Epoch 2/20
85/84 [==============================] - ETA: 0s - loss: 0.3919 - accuracy: 0.8747
Epoch 2: val_loss did not improve from 4.27529
84/84 [==============================] - 161s 2s/step - loss: 0.3919 - accuracy: 0.8747 - val_loss: 4.4531 - val_accuracy: 0.2788
Epoch 3/20
85/84 [==============================] - ETA: 0s - loss: 0.3260 - accuracy: 0.8923
Epoch 3: val_loss did not improve from 4.27529
84/84 [==============================] - 162s 2s/step - loss: 0.3260 - accuracy: 0.8923 - val_loss: 4.5896 - val_accuracy: 0.2732
Epoch 4/20
85/84 [==============================] - ETA: 0s - loss: 0.2738 - accuracy: 0.9083
Epoch 4: val_loss did not improve from 4.

In [ ]:
model.save('./' +  model_name + '.h5')